In [1]:
import os
import sys
import pandas as pd 
import pingouin as pg 

import torch

parent_dir = os.path.dirname(os.getcwd())
src_dir = os.path.join(parent_dir, 'src')


sys.path.insert(0, src_dir)
import read
#import similarity_score


In [96]:
similarity_score = pd.read_csv(os.path.join(parent_dir,'results/cosine_similarity/similarity_score.csv'))

In [47]:
import os
import clip
import torch
from torchvision.datasets import CIFAR100
from PIL import Image

def cosine_similarity(a, b):
    return torch.nn.functional.cosine_similarity(a, b, dim=-1)

# Load the model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)

# Download the dataset

image = Image.open('C:/Users/670302093/OneDrive - Signify/ClipSoundSymbolism/dataset/images/AffectNet_face/Fear/1020.jpg')

image_input = preprocess(image).unsqueeze(0).to(device)
text_inputs = clip.tokenize(f"an afraid face").to(device)

# Calculate features
with torch.no_grad():
    image_features = model.encode_image(image_input)
    text_features = model.encode_text(text_inputs)

# Pick the top 5 most similar labels for the image
#image_features /= image_features.norm(dim=-1, keepdim=True)
#text_features /= text_features.norm(dim=-1, keepdim=True)
#similarity = image_features @ text_features.T

similarity = cosine_similarity(image_features, text_features)

In [99]:
similarity_score

,words,100006.jpg,100019.jpg,100026.jpg,100028.jpg,100032.jpg,100041.jpg,100055.jpg,100057.jpg,100063.jpg,...,11296.jpg,112974.jpg,112988.jpg,113019.jpg,113037.jpg,113047.jpg,113059.jpg,113094.jpg,113100.jpg,113121.jpg
0,afraid,0.207235,0.237840,0.202311,0.249000,0.221047,0.212581,0.226349,0.248297,0.210366,...,0.219739,0.224288,0.240346,0.203194,0.246778,0.212322,0.216005,0.255374,0.210596,0.209446
1,alse,0.237946,0.230293,0.231312,0.254600,0.232371,0.227505,0.236103,0.246493,0.235163,...,0.209503,0.226189,0.231634,0.206933,0.253564,0.233528,0.223813,0.259036,0.247989,0.224501
2,apache,0.204080,0.226235,0.214380,0.238198,0.200953,0.208183,0.217522,0.249416,0.212143,...,0.176627,0.218984,0.198517,0.176982,0.213278,0.185588,0.210440,0.225181,0.237287,0.206753
3,aphid,0.190248,0.180178,0.181310,0.233025,0.174424,0.159537,0.197109,0.210130,0.189609,...,0.126095,0.163449,0.183502,0.160760,0.211079,0.177348,0.169480,0.187860,0.203349,0.199847
4,bale,0.203384,0.218150,0.218406,0.263168,0.183893,0.217301,0.232679,0.233733,0.171184,...,0.174760,0.201117,0.208752,0.138372,0.218806,0.204754,0.249346,0.249641,0.196241,0.186520
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,zel,0.227065,0.224496,0.219197,0.248092,0.217609,0.214034,0.248320,0.241655,0.236456,...,0.186623,0.211011,0.214213,0.199693,0.246292,0.248281,0.213201,0.244423,0.217738,0.226020
336,zerge,0.220478,0.245994,0.225276,0.257894,0.202696,0.212676,0.241367,0.238961,0.216914,...,0.175646,0.202757,0.240037,0.190814,0.239497,0.230426,0.236300,0.245778,0.195738,0.202050
337,zorce,0.234630,0.231388,0.225525,0.258722,0.233205,0.225893,0.248235,0.247563,0.225528,...,0.201196,0.221609,0.231344,0.211473,0.243961,0.234748,0.234151,0.260816,0.237079,0.224174
338,zourse,0.254037,0.240271,0.231199,0.268990,0.229178,0.223350,0.245832,0.246473,0.243336,...,0.197118,0.227942,0.227955,0.218060,0.256002,0.229425,0.234013,0.272244,0.258333,0.231377


In [98]:
similarity_score.rename(columns={'Unnamed: 0':'words'}, inplace=True)

In [19]:
similarity_score.to_csv()

,Unnamed: 0,100006.jpg,100019.jpg,100026.jpg,100028.jpg,100032.jpg,100041.jpg,100055.jpg,100057.jpg,100063.jpg,...,11296.jpg,112974.jpg,112988.jpg,113019.jpg,113037.jpg,113047.jpg,113059.jpg,113094.jpg,113100.jpg,113121.jpg
0,afraid,0.207235,0.237840,0.202311,0.249000,0.221047,0.212581,0.226349,0.248297,0.210366,...,0.219739,0.224288,0.240346,0.203194,0.246778,0.212322,0.216005,0.255374,0.210596,0.209446
1,alse,0.237946,0.230293,0.231312,0.254600,0.232371,0.227505,0.236103,0.246493,0.235163,...,0.209503,0.226189,0.231634,0.206933,0.253564,0.233528,0.223813,0.259036,0.247989,0.224501
2,apache,0.204080,0.226235,0.214380,0.238198,0.200953,0.208183,0.217522,0.249416,0.212143,...,0.176627,0.218984,0.198517,0.176982,0.213278,0.185588,0.210440,0.225181,0.237287,0.206753
3,aphid,0.190248,0.180178,0.181310,0.233025,0.174424,0.159537,0.197109,0.210130,0.189609,...,0.126095,0.163449,0.183502,0.160760,0.211079,0.177348,0.169480,0.187860,0.203349,0.199847
4,bale,0.203384,0.218150,0.218406,0.263168,0.183893,0.217301,0.232679,0.233733,0.171184,...,0.174760,0.201117,0.208752,0.138372,0.218806,0.204754,0.249346,0.249641,0.196241,0.186520


In [20]:
top_10_scores = similarity_score.iloc[:, 1:].apply(lambda row: row.nlargest(10), axis=1)


In [49]:
similarity_score.rename(columns={'Unnamed: 0':'words'}, inplace=True)

In [51]:
len(similarity_score.columns)

3001

In [63]:
means = {}
for i in range(1, similarity_score.shape[1], 500):
    
    means.append(similarity_score.iloc[:, i:i+500].mean(axis=1))

1
501
1001
1501
2001
2501


In [72]:
EMOTION_DICT = {0: 'Anger', 1: 'Disgust', 2: 'Fear', 3: 'Joy', 4: 'Sadness', 5: 'Surprise'}
similarity_mean = {}

for n,mean_list in enumerate(means):

    similarity_mean[EMOTION_DICT[n]] = {}

    for i, mean in enumerate(mean_list):

        similarity_mean[EMOTION_DICT[n]][similarity_score.iloc[i,0]] =  mean



In [77]:
similarity_mean_df = pd.DataFrame.from_dict(similarity_mean)

In [81]:
df_emotion = read.emotions_df()

In [82]:
df_emotion.head()

,IDs,Word,ARPA Pron,Real Word Flag,Joy,Sadness,Anger,Disgust,Fear,Surprise
0,0,afraid,ah f r ey d,1,0.3125,0.8333,0.3333,0.1875,0.6875,0.3333
1,1,alse,ae l s,0,0.6875,0.4375,0.5625,0.4792,0.4375,0.5625
2,2,apache,ah p ae ch iy,1,0.2917,0.6458,0.7708,0.4792,0.5000,0.5833
3,3,aphid,ae f ih d,1,0.3333,0.6250,0.4792,0.5625,0.6042,0.3125
4,4,bale,b ey l,1,0.5000,0.5208,0.4167,0.3542,0.4583,0.0833


In [87]:
similarity_mean_df_new = similarity_mean_df.reset_index().rename(columns={'index': 'Word'})

In [104]:
final_similarity_mean = df_emotion.merge(similarity_mean_df_new, on='Word').drop(columns=['IDs', 'ARPA Pron'])

In [105]:
final_similarity_mean

,Word,Real Word Flag,Joy_x,Sadness_x,Anger_x,Disgust_x,Fear_x,Surprise_x,Anger_y,Disgust_y,Fear_y,Joy_y,Sadness_y,Surprise_y
0,afraid,1,0.3125,0.8333,0.3333,0.1875,0.6875,0.3333,0.223304,0.227507,0.231807,0.213571,0.221866,0.224300
1,alse,0,0.6875,0.4375,0.5625,0.4792,0.4375,0.5625,0.235634,0.240410,0.237847,0.230116,0.231188,0.235187
2,apache,1,0.2917,0.6458,0.7708,0.4792,0.5000,0.5833,0.218303,0.220478,0.215647,0.207442,0.214663,0.210928
3,aphid,1,0.3333,0.6250,0.4792,0.5625,0.6042,0.3125,0.183633,0.197446,0.194168,0.177422,0.187822,0.187904
4,bale,1,0.5000,0.5208,0.4167,0.3542,0.4583,0.0833,0.211642,0.220014,0.216170,0.195840,0.203376,0.206224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,zel,0,0.6667,0.3750,0.5417,0.2083,0.3958,0.7500,0.227364,0.235000,0.231976,0.218837,0.225444,0.227068
336,zerge,0,0.6667,0.3333,0.4375,0.4167,0.4375,0.5625,0.229641,0.232803,0.233577,0.205625,0.219320,0.223590
337,zorce,0,0.4583,0.5833,0.7917,0.6667,0.6250,0.6250,0.234819,0.240208,0.237077,0.224774,0.230412,0.233988
338,zourse,0,0.5625,0.3958,0.5833,0.5208,0.3750,0.6458,0.239539,0.244389,0.244210,0.233524,0.235836,0.240487


In [121]:
dict_words = {'real' : final_similarity_mean[final_similarity_mean['Real Word Flag'] == 1],
              'pseudowords' : final_similarity_mean[final_similarity_mean['Real Word Flag'] == 0]}

In [ ]:
def compute_pearson(df_dict, column):
    pearson_all = pd.DataFrame()
    for img_generator, df in df_dict.items():
        grouped = df.groupby(['classifier', 'type'])

        # Compute Pearson correlation for each group
        for name, group in grouped:
            pearson_df = pg.corr(group['rating.mean'], group[column])
            pearson_df['image generator'] = img_generator
            pearson_df['classifier'] = name[0]
            pearson_df['type'] = name[1]
            pearson_all = pd.concat([pearson_all, pearson_df], ignore_index=True)


In [126]:
pearson_all = pd.DataFrame()
for typ, df in dict_words.items():
    for emotion in EMOTION_DICT.values():

        pearson_df = pg.corr(df[f'{emotion}_x'], df[f'{emotion}_y'])
        pearson_df['emotion'] = emotion
        pearson_df['type'] = typ

        pearson_all = pd.concat([pearson_all, pearson_df], ignore_index=True)



In [127]:
pearson_all

,n,r,CI95%,p-val,BF10,power,emotion,type
0,68,0.280378,"[0.04, 0.49]",0.020566,2.091,0.647139,Anger,real
1,68,0.272289,"[0.04, 0.48]",0.024682,1.79,0.620396,Disgust,real
2,68,0.043297,"[-0.2, 0.28]",0.725903,0.161,0.064030,Fear,real
3,68,-0.196324,"[-0.42, 0.04]",0.108588,0.534,0.364652,Joy,real
4,68,0.254194,"[0.02, 0.46]",0.036460,1.289,0.559106,Sadness,real
5,68,-0.152219,"[-0.38, 0.09]",0.215274,0.321,0.237676,Surprise,real
6,272,0.033615,"[-0.09, 0.15]",0.580954,0.088,0.085563,Anger,pseudowords
7,272,0.039559,"[-0.08, 0.16]",0.515906,0.094,0.099594,Disgust,pseudowords
8,272,-0.007979,"[-0.13, 0.11]",0.895782,0.077,0.051911,Fear,pseudowords
9,272,-0.026904,"[-0.15, 0.09]",0.658671,0.084,0.072609,Joy,pseudowords


In [106]:


pg.corr(final_similarity_mean['Anger_x'], final_similarity_mean['Anger_y'])

,n,r,CI95%,p-val,BF10,power
pearson,340,0.067992,"[-0.04, 0.17]",0.211099,0.148,0.24


In [95]:
final_similarity_mean.to_csv('similarity_score_avg.csv', index=False)

In [56]:
similarity_score.iloc[:, 0:501]

,words,100006.jpg,100019.jpg,100026.jpg,100028.jpg,100032.jpg,100041.jpg,100055.jpg,100057.jpg,100063.jpg,...,10733.jpg,107346.jpg,107364.jpg,107372.jpg,107376.jpg,107378.jpg,107425.jpg,107426.jpg,107439.jpg,107447.jpg
0,afraid,0.207235,0.237840,0.202311,0.249000,0.221047,0.212581,0.226349,0.248297,0.210366,...,0.241020,0.244239,0.208533,0.228002,0.227504,0.223869,0.238907,0.241665,0.211290,0.198419
1,alse,0.237946,0.230293,0.231312,0.254600,0.232371,0.227505,0.236103,0.246493,0.235163,...,0.234932,0.259239,0.236159,0.244995,0.233420,0.246822,0.258545,0.256898,0.208379,0.237251
2,apache,0.204080,0.226235,0.214380,0.238198,0.200953,0.208183,0.217522,0.249416,0.212143,...,0.218800,0.234328,0.179142,0.235907,0.223452,0.225873,0.218522,0.209687,0.185863,0.197483
3,aphid,0.190248,0.180178,0.181310,0.233025,0.174424,0.159537,0.197109,0.210130,0.189609,...,0.211963,0.191342,0.176242,0.226735,0.169716,0.179222,0.209404,0.201550,0.218891,0.128471
4,bale,0.203384,0.218150,0.218406,0.263168,0.183893,0.217301,0.232679,0.233733,0.171184,...,0.244274,0.216072,0.153194,0.226197,0.164755,0.231801,0.236882,0.251013,0.180289,0.209724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,zel,0.227065,0.224496,0.219197,0.248092,0.217609,0.214034,0.248320,0.241655,0.236456,...,0.235390,0.258768,0.209253,0.240002,0.238325,0.240247,0.253150,0.249733,0.195266,0.220927
336,zerge,0.220478,0.245994,0.225276,0.257894,0.202696,0.212676,0.241367,0.238961,0.216914,...,0.247016,0.280782,0.216043,0.211950,0.252090,0.269334,0.247463,0.273276,0.187307,0.251434
337,zorce,0.234630,0.231388,0.225525,0.258722,0.233205,0.225893,0.248235,0.247563,0.225528,...,0.248223,0.260767,0.227807,0.236987,0.227393,0.253500,0.257610,0.267206,0.202119,0.232235
338,zourse,0.254037,0.240271,0.231199,0.268990,0.229178,0.223350,0.245832,0.246473,0.243336,...,0.245058,0.269292,0.237301,0.260589,0.240398,0.243807,0.248848,0.262246,0.209260,0.240564


In [ ]:
similarity_score.drop('words', axis=1).max(axis=1)[:20]

In [16]:
similarity_score.dtypes

words          object
100006.jpg    float64
100019.jpg    float64
100026.jpg    float64
100028.jpg    float64
               ...   
113047.jpg    float64
113059.jpg    float64
113094.jpg    float64
113100.jpg    float64
113121.jpg    float64
Length: 3001, dtype: object

In [17]:
top_10_scores = similarity_score.apply(lambda row: row.nlargest(10), axis=1)

TypeError: Cannot use method 'nlargest' with dtype object